# Example 14.3 Adiabatic Compression and Expansion

A heat pump using the refrigerant R-22 operates with a mass flow rate of 100 kg/hr. The fluid enters the compressor at T1 = 300 K and P1 = 1 bar. The compressor heat loss is neglected. The outlet pressure of the compressor is 5 bar. If the isentropic efficiency of the compressor is 0.7 and the mechanical efficiency is 0.9, what is the power draw of the compressor and how how is the refrigerant when it exits the compressor?

The textbook uses the Peng-Robinson EOS, so to compare, use that as well.

In [1]:
# Set the conditions and imports
from scipy.constants import bar, hour
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CEOSGas, FlashPureVLS
fluid = 'R-22'
constants, correlations = ChemicalConstantsPackage.from_IDs([fluid])

T1 = 300.0
P1 = 1*bar
P2 = 5*bar
eta_isentropic = 0.7
eta_mechanical = 0.9

In [2]:
# Use the default originally published Peng-Robinson models
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas)
liquid = CEOSLiquid(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
gas = CEOSGas(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])

# Flash at inlet conditions to obtain initial enthalpy
state_1 = flasher.flash(T=T1, P=P1)
# Flash at outlet condition - entropy is conserved by compressors and expanders!
state_2_ideal = flasher.flash(S=state_1.S(), P=P2)
# Compute the change in enthalpy
delta_H_ideal = (state_2_ideal.H()-state_1.H())
# The definition of isentropic efficiency means that the actual amount of heat added is
# dH_actual = dH_idea/eta_isentropic
H_added_to_fluid_actual = delta_H_ideal/eta_isentropic

state_2 = flasher.flash(H=state_1.H() + H_added_to_fluid_actual, P=P2)

# To compute the actual power, itis more convinient to use the mass enthalpy
actual_power_per_kg = (state_2.H_mass() - state_1.H_mass())/(eta_mechanical) # W/kg
actual_power = actual_power_per_kg*100/hour
print(f'The actual power is {actual_power:.0f} W')
print(f'The actual outlet temperature is {state_2.T: .2f} K')

The actual power is 2251 W
The actual outlet temperature is  405.98 K


The power given in the textbook is 2257 W and 405.68 K out. No details as to the liquid heat capacity are given. As refrigerants are well defined substances, it is recommended for anyone doing modeling with them to use a high-accuracy model wherever possible.